# Extraction des indices dans les 11 Sites de Contrôle au Sol (SCS) du Centre de Suivi Ecoligique (CSE)

In [1]:
# Import des librarie de geemap
import ee
import geemap

Enter verification code:  4/1Adeu5BUREOzpMWPrua7zSXS8ZUS8OBrOj-GBsIO7-NQLYnEjp5tUjrm-yu0



Successfully saved authorization token.


In [ ]:
# Authentification du compte google earth engine (GEE)
ee.Authenticate()
ee.Initialize()

In [6]:
# Ajouter du ROI à partir de GEE
zsp = ee.FeatureCollection("projects/ee-sambagorguimane/assets/zsp")

# Ajouter une collection d'image GEE
sentinel_2 = ee.ImageCollection("COPERNICUS/S2_SR_HARMONIZED").filter(
    ee.Filter.date('2021-03-25','2021-04-04')
).filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',7)).filterBounds(zsp).median().clip(zsp)

# Paramètres de visualisation 
vis = {
    "min": 0.0,
    "max": 7000,
    "bands": ["B4", "B3", "B2"],
}

# Visualisation 
Map = geemap.Map()
Map.add_basemap('SATELLITE')
Map.centerObject(zsp,7)
Map.addLayer(zsp)
Map.addLayer(sentinel_2, vis, 'Sentinel2')
Map

Map(center=[14.888344806144062, -14.207072727106443], controls=(WidgetControl(options=['position', 'transparen…

# Calcul des indices de végétation

## Visualization parameters

In [10]:
vis_params = {max: 1.0,
    min: -0.5,
    'palette': ['#0000ff', '#008000', '#ff0000',
                '#ffff00', '#5effee','#f7ff9b',
               '#e9bdff','#7dacff','#ffc1a7']}

### 1. Normalized Difference Vegetation Index (NDI5)
#### ndi_5 = (nir-swir1)/(nir+swir1)

In [11]:
def getndi5(image):
    # Calculer le NDI5 à l'aide de cette expression.
    ndi_5 = image.expression ('(nir-swir1)/(nir+swir1)', {
        'nir': image.select ('B8'),  
        'swir1': image.select ('B11')
        }).rename("NDI5")
    image = image.addBands(ndi_5)

    return(image)

# Visualisation NDI5
ndi5 = getndi5(sentinel_2)
ndi_5= ndi5.select(['NDI5'])
map_ndi5 = geemap.Map()
map_ndi5.addLayer(ndi_5, vis_params, 'NDI5')
map_ndi5.centerObject(zsp,7)
map_ndi5

Map(center=[14.888344806144062, -14.207072727106443], controls=(WidgetControl(options=['position', 'transparen…

### 2. Dead Fuel Index (DFI)
#### dfi = 100*(1-swir2/swir1)*(red/nir)

In [12]:
def getdfi(image):
    # Calculer le DFI à l'aide de cette  expression.
    dfi = image.expression ('100*(1-(swir2/swir1))*(red/nir)', {
        'nir': image.select ('B8'),  
        'red': image.select ('B4'),
        'swir1': image.select('B11'),
        'swir2': image.select('B12')
        }).rename("DFI")
    image = image.addBands(dfi)

    return(image)

# Visualisation DFI
dfi = getdfi(sentinel_2)
dfi = dfi.select(['DFI'])
map_dfi = geemap.Map()
map_dfi.addLayer(dfi, vis_params, 'DFI')
map_dfi.centerObject(zsp,7)
map_dfi

Map(center=[14.888344806144062, -14.207072727106443], controls=(WidgetControl(options=['position', 'transparen…

### 3. Transformed CARI (TCARI)
#### tcari = 3*[(nir-red)-0.2*(nir-green)*(nir/red)]

In [13]:
def gettcari(image):
    # Calculer le TCARI à l'aide de cette expression.
    tcari = image.expression ('(3*((nir-red)-0.2*(nir-green)*(nir/red))/10000)', {
        'nir': image.select ('B8'),  
        'red': image.select ('B4'),
        'green': image.select('B3')
        }).rename("TCARI")
    image = image.addBands(tcari)
    return(image)

# Visualisation TCARI
tcari = gettcari(sentinel_2)
tcari = tcari.select(['TCARI'])
map_tcari = geemap.Map()
map_tcari.addLayer(tcari, vis_params, 'TCARI')
map_tcari.centerObject(zsp,7)
map_tcari

Map(center=[14.888344806144062, -14.207072727106443], controls=(WidgetControl(options=['position', 'transparen…

### 4. Green Residue Cover Index (GRCI)
#### grci = (green-blue)/(green+blue)

In [14]:
def getgrci(image):
    # Calculer le GRCI à l'aide de cette expression.
    grci = image.expression ('(green-blue)/(green+blue)', {
        'green': image.select ('B3'),  
        'blue': image.select ('B2')
        }).rename("GRCI")
    image = image.addBands(grci)

    return(image)

# Visualisation GRCI
grci = getgrci(sentinel_2)
grci = grci.select(['GRCI'])
map_grci = geemap.Map()
map_grci.addLayer(grci, vis_params, 'GRCI')
map_grci.centerObject(zsp,7)
map_grci

Map(center=[14.888344806144062, -14.207072727106443], controls=(WidgetControl(options=['position', 'transparen…

### 5. Simple Ratio Index (SRI)
#### sri = nir/red

In [15]:
def getsri(image):
    # Calculer le SRI à l'aide de cette expression.
    sri = image.expression ('nir/red', {
        'nir': image.select ('B8'),  
        'red': image.select ('B4')
        }).rename("SRI")
    image = image.addBands(sri)

    return(image)

# Visualisation SRI
sri = getsri(sentinel_2)
sri = sri.select(['SRI'])
map_sri = geemap.Map()
map_sri.addLayer(sri, vis_params, 'SRI')
map_sri.centerObject(zsp,7)
map_sri

Map(center=[14.888344806144062, -14.207072727106443], controls=(WidgetControl(options=['position', 'transparen…

### 6. Ratio Vegetation Index 2 (RVI2)
#### rvi2 = green/red

In [16]:
def getrvi2(image):
    # Calculer le RVI à l'aide de cette  expression.
    rvi2 = image.expression ('green/red', {
        'green': image.select ('B3'),  
        'red': image.select ('B4')
        }).rename("RVI2")
    image = image.addBands(rvi2)

    return(image)

# Visualisation RVI2
rvi2 = getrvi2(sentinel_2)
rvi2 = rvi2.select(['RVI2'])
map_rvi2 = geemap.Map()
map_rvi2.addLayer(rvi2, vis_params, 'RVI2')
map_rvi2.centerObject(zsp,7)
map_rvi2

Map(center=[14.888344806144062, -14.207072727106443], controls=(WidgetControl(options=['position', 'transparen…

### 7. Crop Residue Cover (CRC)
#### crc = (swir1-green)/(swir1+green)

In [17]:
def getcrc(image):
    # Calculer le CRC à l'aide de cette expression.
    crc = image.expression ('(swir1-green)/(swir1+green)', {
        'swir1': image.select('B11'),
        'green': image.select('B3')
        }).rename("CRC")
    image = image.addBands(crc)

    return(image)

# Visualisation CRC
crc = getcrc(sentinel_2)
crc = crc.select(['CRC'])
map_crc = geemap.Map()
map_crc.addLayer(crc, vis_params, 'CRC')
map_crc.centerObject(zsp,7)
map_crc

Map(center=[14.888344806144062, -14.207072727106443], controls=(WidgetControl(options=['position', 'transparen…

### 8. Ratio Vegetation Index 3 (RVI3)
#### rvi3 = green/blue

In [18]:
def getrvi3(image):
    # Calculer le RVI3 à l'aide de cette expression.
    rvi3 = image.expression ('green/blue', {
        'green': image.select ('B3'),  
        'blue': image.select ('B2')
        }).rename("RVI3")
    image = image.addBands(rvi3)

    return(image)

# Visualisation RVI3
rvi3 = getrvi3(sentinel_2)
rvi3 = rvi3.select(['RVI3'])
map_rvi3 = geemap.Map()
map_rvi3.addLayer(rvi3, vis_params, 'RVI3')
map_rvi3.centerObject(zsp,7)
map_rvi3

Map(center=[14.888344806144062, -14.207072727106443], controls=(WidgetControl(options=['position', 'transparen…

### 9. Normalized Difference Vegetation Index (NDVI)
#### NDVI = (NIR-RED)/(NIR+RED)

In [19]:
def getndvi(image):
    # Calculer le NDVI à l'aide de cette expression.
    ndvi = image.expression ('(nir-red)/(nir+red)', {
        'nir': image.select ('B8'),  
        'red': image.select ('B4')
        }).rename("NDVI")
    image = image.addBands(ndvi)

    return(image)

# Visualisation NDVI
ndvi = getndvi(sentinel_2)
ndvi = ndvi.select(['NDVI'])
map_ndvi = geemap.Map()
map_ndvi.addLayer(ndvi, vis_params, 'NDVI')
map_ndvi.centerObject(zsp,7)
map_ndvi

Map(center=[14.888344806144062, -14.207072727106443], controls=(WidgetControl(options=['position', 'transparen…

### 10. Normalized Difference Index 7 (NDI7)
#### NDV7 = (NIR-SWIR2)/(NIR+SWIR2)

In [20]:
def getndi7(image):
    # Calculer le NDI7 à l'aide de cette expression.
    ndi7 = image.expression ('(nir-swir2)/(nir+swir2)', {
        'nir': image.select ('B8'),  
        'swir2': image.select ('B12')
        }).rename("NDI7")
    image = image.addBands(ndi7)

    return(image)

# Visualisation NDI7
ndi7 = getndi7(sentinel_2)
ndi7 = ndi7.select(['NDI7'])
map_ndi7 = geemap.Map()
map_ndi7.addLayer(ndi7, vis_params, 'NDI7')
map_ndi7.centerObject(zsp,7)
map_ndi7

Map(center=[14.888344806144062, -14.207072727106443], controls=(WidgetControl(options=['position', 'transparen…

### 11. Normalized Difference Senescent Vegetation Index (NDSVI)
#### NDSVI = (SWIR1-RED)/(SWIR1+RED)

In [21]:
def getndsvi(image):
    # Calculer le NDSVI à l'aide de cette expression.
    ndsvi = image.expression ('(SWIR1-RED)/(SWIR1+RED)', {
        'RED': image.select ('B4'),  
        'SWIR1': image.select ('B11')
        }).rename("NDSVI")
    image = image.addBands(ndsvi)

    return(image)

# Visualisation NDSVI
ndsvi = getndsvi(sentinel_2)
ndsvi = ndsvi.select(['NDSVI'])
map_ndsvi = geemap.Map()
map_ndsvi.addLayer(ndsvi, vis_params, 'NDSVI')
map_ndsvi.centerObject(zsp,7)
map_ndsvi

Map(center=[14.888344806144062, -14.207072727106443], controls=(WidgetControl(options=['position', 'transparen…

### 12. Normalized Difference Tillage Index
#### NDTI=(B11-B12)/(B11+B12)

In [22]:
def getndti(image):
    # Calculer le NDTI à l'aide de cette expression.
    ndti = image.expression ('(SWIR1-SWIR2)/(SWIR1+SWIR2)', {
        'SWIR2': image.select ('B12'),  
        'SWIR1': image.select ('B11')
        }).rename("NDTI")
    image = image.addBands(ndti)

    return(image)

# Visualisation NDTI
ndti = getndti(sentinel_2)
ndti = ndti.select(['NDTI'])
map_ndti = geemap.Map()
map_ndti.addLayer(ndti, vis_params, 'NDTI')
map_ndti.centerObject(zsp,7)
map_ndti

Map(center=[14.888344806144062, -14.207072727106443], controls=(WidgetControl(options=['position', 'transparen…

### 13. Blue normalized difference vegetation index
#### BNDVI=((B8-B2))/((B8+B2))

In [23]:
def getbndvi(image):
    # Calculer le BNDVI à l'aide de cette expression.
    bndvi = image.expression ('(nir-blue)/(nir+blue)', {
        'nir': image.select ('B8'),  
        'blue': image.select ('B2')
        }).rename("BNDVI")
    image = image.addBands(bndvi)

    return(image)

# Visualisation BNDVI
bndvi = getbndvi(sentinel_2)
bndvi = bndvi.select(['BNDVI'])
map_bndvi = geemap.Map()
map_bndvi.addLayer(bndvi, vis_params, 'BNDVI')
map_bndvi.centerObject(zsp,7)
map_bndvi

Map(center=[14.888344806144062, -14.207072727106443], controls=(WidgetControl(options=['position', 'transparen…

### 14. Green Normalized Difference Vegetation Index
#### GNDVI=((B8-B3))/((B8+B3))

In [24]:
def getgndvi(image):
    # Calculer le GNDVI à l'aide de cette expression.
    gndvi = image.expression ('(nir-green)/(nir+green)', {
        'nir': image.select ('B8'),  
        'green': image.select ('B3')
        }).rename("GNDVI")
    image = image.addBands(gndvi)

    return(image)

# Visualisation GNDVI
gndvi = getgndvi(sentinel_2)
gndvi = gndvi.select(['GNDVI'])
map_gndvi = geemap.Map()
map_gndvi.addLayer(gndvi, vis_params, 'GNDVI')
map_gndvi.centerObject(zsp,7)
map_gndvi

Map(center=[14.888344806144062, -14.207072727106443], controls=(WidgetControl(options=['position', 'transparen…

### 15. Standardized Normalized Difference Vegetation Index
#### SNDVI=((B8-B4))/((B4+B8+0,16))

In [25]:
def getsndvi(image):
    # Calculer le SNDVI à l'aide de cette expression.
    sndvi = image.expression ('(nir-red)/(nir+red+0.16)', {
        'nir': image.select ('B8'),  
        'red': image.select ('B4')
        }).rename("SNDVI")
    image = image.addBands(sndvi)

    return(image)

# Visualisation SNDVI
sndvi = getsndvi(sentinel_2)
sndvi = sndvi.select(['SNDVI'])
map_sndvi = geemap.Map()
map_sndvi.addLayer(sndvi, vis_params, 'SNDVI')
map_sndvi.centerObject(zsp,7)
map_sndvi

Map(center=[14.888344806144062, -14.207072727106443], controls=(WidgetControl(options=['position', 'transparen…

### 16. Infrared Percentage Vegetation Index
#### IPVI=B8/((B8+B4))

In [26]:
def getipvi(image):
    # Calculer le IPVI à l'aide de cette expression.
    ipvi = image.expression ('(nir)/(nir+red)', {
        'nir': image.select ('B8'),  
        'red': image.select ('B4')
        }).rename("IPVI")
    image = image.addBands(ipvi)

    return(image)

# Visualisation IPVI
ipvi = getipvi(sentinel_2)
ipvi = ipvi.select(['IPVI'])
map_ipvi = geemap.Map()
map_ipvi.addLayer(ipvi, vis_params, 'IPVI')
map_ipvi.centerObject(zsp,7)
map_ipvi

Map(center=[14.888344806144062, -14.207072727106443], controls=(WidgetControl(options=['position', 'transparen…

### 17. Green Vegetation Index
#### VIgreen=((B3-B4))/((B3+B4))

In [27]:
def getvigreen(image):
    # Calculer le VIgreen à l'aide de cette expression.
    vigreen = image.expression ('(green-red)/(green+red)', {
        'green': image.select ('B3'),  
        'red': image.select ('B4')
        }).rename("VIgreen")
    image = image.addBands(vigreen)

    return(image)

# Visualisation VIgreen
vigreen = getvigreen(sentinel_2)
vigreen = vigreen.select(['VIgreen'])
map_vigreen = geemap.Map()
map_vigreen.addLayer(vigreen, vis_params, 'VIgreen')
map_vigreen.centerObject(zsp,7)
map_vigreen

Map(center=[14.888344806144062, -14.207072727106443], controls=(WidgetControl(options=['position', 'transparen…

# Spatialisation des paramètres de qualité
## 1. Fourrage herbacé

## 1.1. Fibre au Détergent Acide (ADF)

In [91]:
# application de l'équation de ADF
ADF = grci.multiply(-141.51).add(ndsvi.multiply(55.38)).subtract(rvi2.multiply(33.60)).add(86.61).rename("ADF_QUALITY")

In [93]:
# Visualisation de l'ADF
map_adf = geemap.Map()
map_adf.addLayer(image_clipped, vis_params, 'ADF')
map_adf.centerObject(zsp,7)
map_adf

Map(center=[14.88834476838222, -14.207072748536515], controls=(WidgetControl(options=['position', 'transparent…

In [102]:
# Exporter l'image dans le drive
geemap.ee_export_image_to_drive(
    image=image_clipped, description='ADF_quality_M2', folder='quality_forage', region=zsp, scale=10)

## 1.2. Matière Azotée Totale (MAT)

In [50]:
# application de l'équation de MAT
MAT = ipvi.multiply(-23.68).subtract(vigreen.multiply(14.89)).add(15.07).rename("MAT_QUALITY")

In [51]:
# Visualisation de la MAT
map_mat = geemap.Map()
map_mat.addLayer(MAT, vis_params, 'MAT')
map_mat.centerObject(zsp,7)
map_mat

Map(center=[14.888344806144062, -14.207072727106443], controls=(WidgetControl(options=['position', 'transparen…

In [52]:
# Exporter l'image dans le drive
geemap.ee_export_image_to_drive(
    MAT, description='MAT_quality_M2', folder='quality_forage', region=zsp, scale=10
)

## 1.3. Digestibilité en Matière Sèche (SMO)

In [53]:
# application de l'équation de SMO
SMO = grci.multiply(91.13).subtract(ndsvi.multiply(41.22)).add(19.46).rename("SMO_QUALITY")

In [54]:
# Visualisation de la SMO
map_smo = geemap.Map()
map_smo.addLayer(SMO, vis_params, 'SMO')
map_smo.centerObject(zsp,7)
map_smo

Map(center=[14.888344806144062, -14.207072727106443], controls=(WidgetControl(options=['position', 'transparen…

In [55]:
# Exporter l'image dans le drive
geemap.ee_export_image_to_drive(
    SMO, description='SMO_quality_M2', folder='quality_forage', region=zsp, scale=10
)

## 1.4. Cellulose Brute (CBW)

In [56]:
# application de l'équation de CBW
CBW = tcari.multiply(13.56).subtract(grci.multiply(64.29)).add(59.81).rename("CBW_QUALITY")

In [57]:
# Visualisation de la CBW
map_CBW = geemap.Map()
map_CBW.addLayer(CBW, vis_params, 'CBW')
map_CBW.centerObject(zsp,7)
map_CBW

Map(center=[14.888344806144062, -14.207072727106443], controls=(WidgetControl(options=['position', 'transparen…

In [82]:
# Exporter l'image dans le drive
geemap.ee_export_image_to_drive(
    CBW, description='CBW_quality_M2', folder='quality_forage', region=zsp, scale=10
)

## 2. Fourrage ligneux

## 2.1. La Fibre au Détergent Acide (ADF)

In [ ]:
# Application de l'équation de l'ADF
adf_quality_f = dfi.multiply(-0.6722).add(ndvi.multiply(174.2272)).subtract(grci.multiply(47.5443)).subtract(ndsvi.multiply(133.8579)).subtract(tcari.multiply(160.8079)).add(83.5918).rename("ADF_QUALITY_F");

In [ ]:
# Visualisation de l'ADF
map_adf_f = geemap.Map()
map_adf_f.addLayer(image_clipped, vis_params, 'ADF_F')
map_adf_f.centerObject(zsp,7)
map_adf_f

In [ ]:
# Exporter l'image dans le drive
geemap.ee_export_image_to_drive(
    image=image_clipped, description='ADF_quality_F_M2', folder='quality_forage', region=zsp, scale=10)

## 2.2. La Matière Azotée Totale (MAT)

In [ ]:
# Application de l'équation de la MAT
mat_quality_f = ndi_5.multiply(-45.82).subtract(ndsvi.multiply(27.71)).add(tcari.multiply(49.40)).subtract(rvi2.multiply(26.90)).add(26.11).rename("MAT_QUALITY_F");

In [ ]:
# Visualisation de la MAT
map_mat_f = geemap.Map()
map_mat_f.addLayer(CBW, vis_params, 'MAT_F')
map_mat_f.centerObject(zsp,7)
map_mat_f

In [ ]:
# Exporter l'image dans le drive
geemap.ee_export_image_to_drive(
    image=image_clipped, description='MAT_quality_F_M2', folder='quality_forage', region=zsp, scale=10)